# 11. 심층 신경망 훈련하기

훨씬 더 복잡한 문제를 해결하기 위해서는 더 깊은 신경망을 만들어야 한다.
하지만 심층 신경망을 만드는 것은 어려운 일이다.

- **그레이디언트 소실**이나 **그레이디언트 폭주** 문제를 만날 수 있다.
- 대규모 신경망을 만들기 위한 훈련 데이터가 충분하지 않거나 레이블을 만드는 작업에 비용이 너무 많이 들 수 있다.
- 훈련이 극단적으로 느려질 수 있다.
- 과대적합될 위험이 크다

## 11.1 그레이디언트 소실과 폭주 문제
역전파 알고리즘은 출력층에서 입력층으로 오차 그래디언트를 전파하면서 진행된다.
그런데 알고리즘이 하위층으로 진행될수록 그래디언트가 점점 작아지는 경우가 많다. 이를 **그레이디언트 소실**이라고 한다.
반대로 그래디언트가 점점 커져서 비정상적으로 큰 가중치로 갱신되는 것을 **그레이디언트 폭주**라고 한다.

### 11.1.1 글로럿과 He 초기화
Xavier Glorot과 Yoshua Bengio가 2010년 발표한 논문에서 불안정한 그레이디언트 문제를 크게 완화하는 방법을 제안한다.

- 예측을 할 때는 정방향, 그레이디언트를 역전파할때는 역방향으로 양방향 신호가 적절하게 흘러야 한다.
- 신호가 죽거나 폭주 또는 소멸하지 않아야 한다.

저자들은 적절한 신호가 흐르기 위해서는 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다고 주장한다.

fan-in과 fan-out의 개수가 같지 않으면 이를 보장할 수 없다. 하지만 글로럿과 벤지오는 실용적인 방법을 제안한다.
각 층의 연결 가중치를 무작위로 초기화하는 것이다.

평균이 0이고 분산이 $ \sigma^2 = \frac{1}{fan_{avg}} $인 정규분포

또는 $ r=\sqrt{\frac{3}{fan_{avg}}} $일 때 $-r$과 $+r$사이의 균등분포

이러한 초기화 전략을 **세이비어 초기화** 또는 **글로럿 초기화**라고 한다.

| 초기화 전략 | 활성화 함수 | 정규분포 |
|--|--|--|
|글로럿|None, tanh, logistic, softmax | $\frac{1}{fan_{avg}}$ |
|He|ReLU와 그 변종들 | $\frac{2}{fan_{in}}$ |
|르쿤|SELU | $\frac{1}{fan_{in}}$ |

케라스는 기본적으로 균등 분포의 글로럿 초기화를 사용한다.

In [1]:
from tensorflow import keras
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

### 11.1.2 수렴하지 않는 활성화 함수
활성화 함수를 잘못 선택하면 그래디언트의 소실이나 폭주로 이어질 수 있다.
처음에는 시그모이드 함수가 최선의 선택일 것이라고 생각했다. 하지만 다른 함수가 훨씬 더 잘 작동한다는 것을 밝혀낸다.
특히 ReLU 함수는 특정 양숫값에 수렴하지 않는다는 장점이 있다.

하지만 ReLU 함수도 단점이 있다. 훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는다는 것이다.
이는 **죽은 ReLU**로 알려져 있다.

이 문제를 해결하기 위해 LeakyReLU같은 변종을 사용한다. 최근 한 논문에서 여러 ReLU 함수의 변종을 비교해본
결과 LeakyReLU가 ReLU보다 항상 성능이 높다는 결론을 얻었다.


In [2]:
def LeakyReLU(a, z):
    return max(a * z, z)

이 논문에서는 훈련하는 동안 주어진 범위에서 $a$를 무작위로 선택하고 테스트 시에는 평균을 사용하는 RReLU와
$a$가 훈련하는 동안 학습되는 PReLU도 비교했다.

툐르크-아르네 클레베르트 등의 논문에서는 ELU라는 새로운 활성화 함수를 제안했다.
이 함수는 저자들의 실험에서 다른 모든 ReLU 변종의 성능을 앞질렀다.

In [3]:
import numpy as np

def ELU(a, z):
    if z < 0:
        return a * (np.exp(z) - 1)
    else:
        return z

이 함수는 몇 가지를 제외하고는 ReLU와 비슷하다.

- $z<0$일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 가까워진다.
이는 그레이디언트 소실 문제를 완화해 준다.
- $z<0$이어도 그레이디언트가 0이 아니므로 죽은 뉴런을 만들지 않는다.
- $a=1$이면 이 함수는 $z=0$에서 급격히 변동하지 않으므로 $z=0$을 포함해
모든 구간에서 매끄러워 경사하강법의 속도를 높여준다.

ELU 함수의 단점은 지수 함수를 사용하므로 ReLU나 그 변종들보다 계산이 느리다는 것이다.

### 11.1.3 배치 정규화
ELU와 함께 He 초기화를 사용하면 훈련 초기 단계에서 그레이디언트 소실이나 폭주 문제를 크게 감소시킬 수
있지만, 훈련하는 동안 다시 발생하지 않으리라는 보장이 없다.

Sergry Ioffe와 Christian Szegedy가 2015년에 그레이디언트 소실과 폭주 문제를 해결하기 위한
**배치 정규화** 기법을 제안했다. 이 기법은 각 층에서 활성화 함수를 통과하기 전이나 후에
모델 연산을 하나 추가하는 것이다.

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

저자들은 활성화 함수의 이후보다 이전에 배치 정규화 층을 추가하는 것이 좋다고 조언한다.
하지만 작업에 따라 선호되는 방식이 다르기 때문에 두 가지 모두 시험해 보고 어떤 것이 데이터셋에
잘 맞는지 확인해야 한다.